#Put your Google Colab link here:
*your link here*

## Important notice: any use of generative AI for completing the assignment is strictly prohibited.

## Get access to a GPU:
To gain access to the GPUs on Colab, navigate to the `Runtime` tab above and select `Change runtime type`.

In [ ]:
# use if working in colab
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


### Warning: to ensure the reproducibility of your results and to achieve the full grade, do not change or remove RANDOM_STATE variables and setting random seed statements. If you remove or change them, you may not get the full grade.

In [ ]:
random_state = 42

## Part 1: Data Loading and Preprocessing (1 pt)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/Shared drives/ECE477 datasets/Assignment8/coviddeep.zip" -d "."

In [ ]:
import os
import numpy as np

path_to_real_data = 'three-way_classification_real_data'

# Loading the data
X_train, y_train, X_validation, y_validation, X_test, y_test = """TO DO"""
print(X_train.shape, X_validation.shape, X_test.shape)
print(np.unique(y_train))

## Part 2: Training (5 pts)

### Part 2.1: Neural Network Architecture (1 pt)
finish DNN class defination


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Network and function definitions
class DNN(torch.nn.Module):
    def __init__(self, D_in, H_1, H_2, H_3, D_out):
        super(DNN, self).__init__()
        # define 4 linear layers
        """TO DO"""

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = F.leaky_relu(self.fc3(x))
        y_pred = (self.fc4(x))
        return y_pred

    def get_embeddings(self, x):
        # get the embedding right before the last linear layer
        """TO DO"""


### Part 2.2: Train DNN (3 pts)


In [ ]:
import random

# Random states
random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
if device == 'cuda':
  torch.cuda.manual_seed(random_state)
  torch.cuda.manual_seed_all(random_state)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False


# Convert training and validation data into torch tensors, move to device
# Remember to set correct data type
"""TO DO"""

# define D_in and D_out
D_in = """TO DO"""
D_out = """TO DO"""

# define DNN parameters
H_1 = 256
H_2 = 128
H_3 = 128

# create DNN model, move to device
"""TO DO"""

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# training loop
num_epochs = 2000
train_accs = []
val_accs = []

best_val_acc = 0.0  # Track highest validation accuracy
best_model_path = """TO DO"""

print("Training loop...")
for t in range(num_epochs):
    # Training
    """TO DO"""

    # Compute training accuracy
    """TO DO"""
    train_accuracy = """TO DO"""
    train_accs.append(train_accuracy)

    # Print loss, accuracies every 100 epochs
    if (t + 1) % 100 == 0:
        model.eval()  # Set model to evaluation mode
        with torch.no_grad():
            # Compute validation accuracy
            """TO DO"""
            val_accuracy = """TO DO"""
            val_accs.append(val_accuracy)

        print(f"Epoch {t+1}/{num_epochs}, Loss: {loss.item():.4f}, Train Acc: {train_accuracy * 100:.2f}%, Val Acc: {val_accuracy * 100:.2f}%")

        # Save the model to best_model_path if validation accuracy is the highest, update best accuracy
        if val_accuracy > best_val_acc:
            """TO DO"""
            print(f"New best model saved at epoch {t+1} with Val Acc: {best_val_acc * 100:.2f}%")

        model.train()  # Switch back to training mode


### Part 2.3: Evaluate model (1 pt)


In [ ]:
# Load the best model, move to device, set to evaluation mode
"""TO DO"""

print(f"Loaded best model from {best_model_path} for evaluation.")

# Convert test data into torch tensors, correct data type, move to device
"""TO DO"""

# Evaluate on test set
with torch.no_grad():
    """TO DO"""

print(f"Test Accuracy of the Best Model: {test_accuracy:.4f}")


## Part 3: CONFINE implementation (12 pts)
Please read CONFINE paper for details.
- Read Algorithm 1 and section 3.2 to understand CONFINE, CONFINE-classwise, and calibration set.
- Read section 4.2 to understand Evaluation Metrics

Note: There's one thing different from the paper --- We don't append a softmax layer in this assignment.

### Part 3.1: prepare data (2 pts)

- In order to improve model accuracy, we will remove training samples that are incorrectly classified by the pre-trained network
- Feature embeddings for the processed training set, the calibration set, and the test set are extracted using the outputs from a certain layer of the pre-trained neural network. Here we use the embedding right before the final fully-connected layer.
- Use validation set as calibration set.




In [ ]:
# Convert training data to torch tensors, ...
"""TO DO"""

# Make predictions on the training set
with torch.no_grad():
    # get predictions
    # Find the indices of correctly predicted samples
    """TO DO"""
    correct_indices = """TO DO"""

# Create new dataset with only correctly predicted points
X_train_correct = """TO DO"""
y_train_correct = """TO DO"""

# Convert X_train_correct, X_validation, X_test into torch tensors, ...
"""TO DO"""

# Get embeddings using the model's get_embeddings() function
# move to cpu, convert to numpy
with torch.no_grad():
    embeddings_train = """TO DO"""
    embeddings_cal = """TO DO"""
    embeddings_test = """TO DO"""

print(embeddings_test.shape)

### Part 3.2: conformal prediction method (8 pts)
Define functions for conformal prediction.

In [ ]:
def A_k(Ut, Yt, u, y, k=5):
    """
    Computes the CONFINE nonconformity score using cosine distance.
    - Ut: embeddings_train, the reference.
    - Yt: y_train, the reference.
    - u: test embedding.
    - y: class label under consideration.
    - k: number of nearest neighbors.

    Returns:
    - A_k score = (average cosine distance of k nearest neighbors with same label) /
                  (average cosine distance of k nearest neighbors with different label)
    """

    # Compute cosine distances to all training embeddings
    cos_distances = np.zeros(len(Ut))  # Initialize the array
    for i in range(len(Ut)):
        # compute cosine distance between Ut[i] and u
        cos_distances[i] = """TO DO"""

    # Separate distances based on class labels (whether Yt is the same with y)
    same_class_distances = """TO DO"""
    diff_class_distances = """TO DO"""

    # Get the mean distance of k smallest distances in each group
    avg_same_class_dist = np.mean("""TO DO""")
    avg_diff_class_dist = np.mean("""TO DO""")

    return avg_same_class_dist / avg_diff_class_dist

def compute_nonconformity_cal(embeddings_train, y_train, embeddings_cal, y_cal, k=5):
    """
    Computes nonconformity scores for the calibration set.
    - embeddings_train: Ut, the reference.
    - y_train: Yt, the reference.
    - embeddings_cal: Uc, embeddings from calibration set.
    - y_cal: Yc, labels from calibration set.
    - k: Number of nearest neighbors.
    """
    alpha_c = np.zeros(len(y_cal))
    for i in range(len(y_cal)):
        # compute Ak((Ut, Yt), ui, yi), where (ui, yi)∈(Uc, Yc)
        alpha_c[i] = """TO DO"""
    return alpha_c

def compute_p_value(embeddings_train, y_train, y_cal, test_embedding, alpha_c, k=5, classwise=False):
    """
    Implements the CONFINE Algorithm for a given test embedding.
    - embeddings_train: Ut, embeddings from training set.
    - y_train: Yt, labels from training set.
    - y_cal: Yc, labels from calibration set.
    - test_embedding: u, embedding of test sample.
    - k: Number of nearest neighbors.
    - classwise: Whether to use CONFINE-classwise computation.

    Return:
    - p_value: Dictionary where keys are class labels, and values are the p-values for each class.
    """
    unique_classes = np.unique(y_train)

    p_value = {}

    # Compute nonconformity scores and p-values for each class, based on the test embedding
    for y_j in unique_classes:
        # compute Ak((Ut, Yt), u, y_j)
        alpha_test = """TO DO"""

        if classwise:
            # Use only calibration samples from class y_j
            mask = """TO DO"""
        else:
            # Use all calibration samples
            mask = """TO DO"""
        # p_value for y_j: p(y_j)
        p_value[y_j] = """TO DO"""


    return p_value


def compute_predictions(p_value, epsilon=0.05):
    """
    Computes the prediction set, prediction, credibility, and confidence based on the p-values.

    - p_value: Dictionary where keys are class labels, and values are the p-values for each class.
    - epsilon: Significance level threshold for determining which classes belong to the prediction set.

    Returns:
    - prediction_set: List of class labels that meet the threshold for inclusion in the prediction set (p-value > epsilon).
    - prediction: The class label with the highest p-value.
    - credibility: The highest p-value corresponding to the final prediction.
    - confidence: 1 - second-highest p-value
    """
    prediction_set = """TO DO"""

    # Sort p_value by the p-value of each class
    sorted_p = """TO DO"""
    prediction = """TO DO"""  # Class with highest p-value
    credibility = """TO DO""" # Highest p-value


    confidence = """TO DO""" # 1 - second highest p-value
    return prediction_set, prediction, credibility, confidence


def evaluate(p_values, y_test, epsilon=0.05):
    """
    Evaluates the performance of the model based on p-values for each test sample.

    - p_values: A list of dictionaries containing p-values for each class, one for each test sample.
    - y_test: Ground truth labels for the test dataset.
    - epsilon: Significance level threshold used to form the prediction set.

    The function computes and prints three evaluation metrics:
    - Accuracy: Proportion of correct predictions.
    - Correct Efficiency: Proportion of predictions where the true class is the only class in the prediction set.
    - Coverage: Proportion of predictions where the true class is included in the prediction set.
    """
    accuracy = 0
    correct_efficiency = 0
    coverage = 0
    for y, p_value in zip(y_test, p_values):
        prediction_set, prediction, credibility, confidence = """TO DO"""
        # update accuracy, correct_efficiency, coverage
        """TO DO"""

    print(f"Accuracy: {accuracy / len(y_test):.4f}")
    print(f"Correct Efficiency: {correct_efficiency / len(y_test):.4f}")
    print(f"Coverage: {coverage / len(y_test):.4f}")
    return


### Part 3.3: Run CONFINE on test dataset (2 pts)
Use previous defined functions. Use k=5, epsilon=0.1.

In [ ]:
# Compute alpha_c for calibration set, use k=5
# Please check the dimentions of input data before you use the functions
# this may take 5 mins
"""TO DO"""
alpha_c = """TO DO"""


In [ ]:
# Compute p_values of test set, using CONFINE
# this may take 15 mins
p_values = []
for test_embedding in embeddings_test:
    p_value = """TO DO"""
    p_values.append(p_value)

# evaluate the result using evaluate(), set epsilon=0.1
"""TO DO"""

In [ ]:
# Compute p_values of test set, using CONFINE-classwise
# this may take 15 mins
p_values_class = []
for test_embedding in embeddings_test:
    p_value = """TO DO"""
    p_values_class.append(p_value)

# evaluate the result using evaluate(), set epsilon=0.1
"""TO DO"""

## Part 4: Questions (2 pts)

### 1. Compare the performance of DNN, CONFINE, and CONFINE-classwise methods. Discuss any perfomance difference observed.

your answer here

### 2. How do Accuracy, Correct Efficiency, and Coverage change as epsilon increases from 0 to 1? Explain the reason.

your answer here